# 個人課題１

In [36]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import json

In [37]:
path = ''
db_name = 'repository.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # テーブルが既に存在するかチェック
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='repository';")
    table_exists = cur.fetchone() is not None

    if table_exists:
        # 既存テーブルをクリア（データをすべて削除）
        cur.execute("DELETE FROM repository;")
        conn.commit()
        print("既存のテーブルをクリアしました")
    else:
        # テーブルを作成
        sql = 'CREATE TABLE repository (id INT, name TEXT , language TEXT, stars INT);'
        cur.execute(sql)
        conn.commit()
        print("テーブルを作成しました")

except sqlite3.Error as e:
    print(f"エラーが発生しました: {e}")

finally:
    # DBへの接続を閉じる
    conn.close()

既存のテーブルをクリアしました


In [38]:
path = ''
db_name = 'repository.db'

# スクレイピング（URL パラメータでページング）
base_url = "https://github.com/orgs/google/repositories"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

page = 1
total_repos_fetched = 0
total_repos_this_page = 0

print("=== Google Organization の全リポジトリを取得開始 ===\n")

while True:
    # URL にページパラメータを指定
    url = base_url if page == 1 else f"{base_url}?page={page}"
    
    print(f"ページ {page} を取得中... (URL: {url[:80]}...)")
    response = requests.get(url, headers=headers)
    print(f"ステータスコード: {response.status_code}")

    #　ステータスコードが200以外の場合に例外を発生させる
    response.raise_for_status()

    soup = BeautifulSoup(response.content, "html.parser")

    # JSON データから抽出（より安定的）
    script_tag = soup.find("script", attrs={"data-target": "react-app.embeddedData"})
    if not script_tag:
        print(f"ページ {page}: embeddedData スクリプトタグが見つかりません。スクレイピング終了")
        break

    # JSON データの抽出
    data = json.loads(script_tag.string)

    # repositories はここに入っている（GitHub の構造）
    repo_list = data["payload"]["orgReposPageRoute"]["repositories"]
    
    if not repo_list:
        print(f"ページ {page}: リポジトリが見つかりません。スクレイピング終了")
        break

    print(f"このページで取得したリポジトリ数: {len(repo_list)}")
    total_repos_this_page = len(repo_list)

    for repo_item in repo_list:
        repo_name = repo_item["name"]
        repo_language = repo_item["primaryLanguage"]["name"] if repo_item.get("primaryLanguage") else "Unknown"
        repo_stars = repo_item.get("starsCount", 0)

        # サーバーに負荷をかけないように0.1秒待機
        time.sleep(0.1)

        try:
            # DB接続オブジェクトの作成
            conn = sqlite3.connect(path + db_name)

            # SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
            cur = conn.cursor()

            # データ挿入SQL
            sql = "INSERT INTO repository (id, name, language, stars) VALUES (?, ?, ?, ?);"

            # SQL文の実行
            cur.execute(sql, (None, repo_name, repo_language, repo_stars))

            # 変更をDBに反映させる
            conn.commit()
            total_repos_fetched += 1

        except sqlite3.Error as e:
            print(f"  エラーが発生しました ({repo_name}): {e}")

        finally:
            # DBへの接続を閉じる
            conn.close()

    # 次ページの存在を確認
    # リポジトリ数が30未満の場合は最後のページと判定
    if total_repos_this_page < 30:
        print(f"\nページ {page}: リポジトリ数が30未満。スクレイピング終了")
        break

    page += 1
    print(f"合計 {total_repos_fetched} 個取得...\n")

print(f"\n=== スクレイピング完了 ===")
print(f"合計 {total_repos_fetched} 個のリポジトリをデータベースに保存しました")

=== Google Organization の全リポジトリを取得開始 ===

ページ 1 を取得中... (URL: https://github.com/orgs/google/repositories...)
ステータスコード: 200
このページで取得したリポジトリ数: 30
ステータスコード: 200
このページで取得したリポジトリ数: 30
合計 30 個取得...

ページ 2 を取得中... (URL: https://github.com/orgs/google/repositories?page=2...)
合計 30 個取得...

ページ 2 を取得中... (URL: https://github.com/orgs/google/repositories?page=2...)
ステータスコード: 200
このページで取得したリポジトリ数: 30
ステータスコード: 200
このページで取得したリポジトリ数: 30
合計 60 個取得...

ページ 3 を取得中... (URL: https://github.com/orgs/google/repositories?page=3...)
合計 60 個取得...

ページ 3 を取得中... (URL: https://github.com/orgs/google/repositories?page=3...)
ステータスコード: 200
このページで取得したリポジトリ数: 30
ステータスコード: 200
このページで取得したリポジトリ数: 30
合計 90 個取得...

ページ 4 を取得中... (URL: https://github.com/orgs/google/repositories?page=4...)
合計 90 個取得...

ページ 4 を取得中... (URL: https://github.com/orgs/google/repositories?page=4...)
ステータスコード: 200
このページで取得したリポジトリ数: 30
ステータスコード: 200
このページで取得したリポジトリ数: 30
合計 120 個取得...

ページ 5 を取得中... (URL: https://github.com/orgs/google/repositori